In [10]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
)
from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
plates = [
    "799_20211001",
    "803_20211012",
    "809_20211012",
    "907_20220321",
    "943_20220322",
    "403_20221005",
    "408_20221013",
    "88_20210915",
    "790_20210907",
    "69_20201119",
    "94_20201123",
    "59_20201126"
]
directory_targ = directory_project
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

analysed:   0%|          | 0/1793 [00:00<?, ?it/s]

In [12]:
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(["94_20201123"])]
analysis_folders

folder_analysis   version  \
36  Analysis_94_20201123_166_Version1  Version1   

                                           total_path Plate strain  \
36  /scratch-shared/amftrack/stitch_temp2/20201123...    94   'A5'   

        medium   CrossDate                date    unique_id  \
36  '001P100N'  '20201123'  23.11.2020, 12:53:  94_20201123   

               datetime  ...  \
36  2020-11-23 12:53:00  ...   

                                 path_time_hypha_info  \
36  Analysis_94_20201123_166_Version1/time_hypha_info   

                                 path_time_plate_info  \
36  Analysis_94_20201123_166_Version1/time_plate_i...   

                               path_global_plate_info   date_run_analysis  \
36  Analysis_94_20201123_166_Version1/global_plate...  01.01.1970, 01:00:   

   PrincePos      root split Pbait Temperature  Temp  
36        39  'Carrot'   'Y'   nan         nan  '25'  

[1 rows x 25 columns]

In [13]:
from amftrack.pipeline.launching.run_super import run_parallel_post

time = "5:00:00"
list_f = [plot_betweenness_random]


list_args = [[]] * len(list_f)
overwrite = True
num_parallel = 6
run_parallel_post(
    "exp_plot.py",
    list_f,
    list_args,
    [directory_targ, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "plot_bet",
    cpus=128,
    name_job='plot_bet_rand',
    node="fat",
)

In [14]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/skeleton_realigned_compressed.mat"] == True]
num_parallel = 128
time = "12:00:00"
skip = False
resolution = 10
args = [directory_targ, skip, resolution]
run_parallel(
    "extract_betweeness.py",
    args,
    folders,
    num_parallel,
    time,
    "betweenness",
    cpus=128,
    node="fat",
    name_job='bet',
)

In [15]:
folders = all_folders.loc[all_folders["unique_id"]=='94_20201123']
folders = folders.sort_values(by="datetime")
folders = folders.iloc[80:82]

exp = Experiment(directory_targ)
exp.load(folders)
exp.dates.sort()
for t in range(len(folders)):
    exp.load_tile_information(t)

2020-11-26 21:09:00
2020-11-26 22:09:00


In [16]:
load_graphs(exp,directory_targ)
exp.save_location = ''

load_study_zone(exp)


In [ ]:
betweenessses = list(current_flow_betweeness.values())
np.max(betweenessses),np.mean(betweenessses)

In [ ]:
betweenessses = list(current_flow_betweeness.values())
betweenessses = [betweeness for betweeness in betweenessses if betweeness>1e-7]
np.max(betweenessses),np.mean(betweenessses)
fig,ax = plt.subplots()
ax.hist(np.log10(betweenessses))

In [17]:
edges = get_all_edges(exp, t)
nodes = get_all_nodes(exp, t)

In [20]:
edges[0].current_flow_betweeness

AttributeError: 'Edge' object has no attribute 'current_flow_betweeness'

In [ ]:
nodes_source = [node for node in nodes if not is_in_study_zone(node,t,1000,150)[1] and is_in_study_zone(node,t,1000,150)[0]]

In [ ]:

nodes_sink = [node for node in nodes if is_in_study_zone(node,t,1000,150)[1] and node.degree(t)==1]

In [ ]:
weights = {(edge.begin.label,edge.end.label) : edge.length_um(t) for edge in edges}
nx.set_edge_attributes(exp.nx_graph[t] , weights, "length")


In [ ]:
G = exp.nx_graph[t]
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
len_connected=[len(nx_graph.nodes) for nx_graph in S]
G_connected = S[np.argmax(len_connected)]
source = [node.label for node in nodes_source if node.label in G_connected]
sink = [node.label for node in nodes_sink if node.label in G_connected]
current_flow_betweeness = nx.edge_current_flow_betweenness_centrality_subset(G_connected,source,sink,weight = "length")
betweeness = nx.edge_betweenness_centrality_subset(G_connected,source,sink,normalized=True,weight = "length")
for edge in exp.nx_graph[t].edges:
    if edge not in current_flow_betweeness.keys() and (edge[1],edge[0]) not in current_flow_betweeness.keys():
        current_flow_betweeness[edge] = 0
        betweeness[edge] = 0        
nx.set_edge_attributes(exp.nx_graph[t] , current_flow_betweeness, "current_flow_betweenness")
nx.set_edge_attributes(exp.nx_graph[t] , betweeness, "betweenness")
betweenessses = list(current_flow_betweeness.values())
np.mean(betweenessses),np.max(betweenessses)
nodes = [node for node in nodes_source if node.label in G_connected]

In [ ]:
betweenessses = list(current_flow_betweeness.values())
vmax = np.log10(np.max(betweenessses))
vmax

In [ ]:
betweenessses = [edge.current_flow_betweeness(t) for edge in edges]
betweenessses = [betweeness for betweeness in betweenessses if betweeness>1e-5]
np.max(betweenessses),np.mean(betweenessses)
fig,ax = plt.subplots()
ax.hist(np.log10(betweenessses))

In [19]:
plt.close("all")
t = 1 
# betweenessses = list(current_flow_betweeness.values())
vmax = -2.5
vmin = -4.5

plot_edge_color_value(exp,t,lambda edge : np.log10(edge.current_flow_betweeness(t)),cmap = cm.get_cmap("Reds", 100),v_min =vmin,v_max = vmax,plot_cmap=True,show_background=False,dilation =10,label_colorbar = "log random walk edge betweeness centrality")

AttributeError: 'Edge' object has no attribute 'current_flow_betweeness'

In [ ]:
betweenessses = [edge.betweeness(t) for edge in edges]
betweenessses = [betweeness for betweeness in betweenessses if betweeness>0]
np.max(betweenessses),np.mean(betweenessses)
fig,ax = plt.subplots()
ax.hist(np.log10(betweenessses))

In [ ]:
vmax = -2.2
vmin = -4.5
plot_edge_width(exp,t,lambda edge : np.log10(edge.betweeness(t)),cmap = cm.get_cmap("Reds", 100),v_min =vmin,v_max = vmax,plot_cmap=True,show_background=False,dilation =10)